In [12]:
with open('07.txt', 'r') as file:
    data = [int(s) for s in file.read().split(',')]


In [13]:
class Halt(Exception):
    pass


class Yield(Exception):
    pass


class VM:
    def __init__(self, memory, inputs, outputs):
        self.ops = {1: self.add, 
                    2: self.mul, 
                    3: self.input_, 
                    4: self.output, 
                    5: self.jnz,
                    6: self.jz,
                    7: self.lt,
                    8: self.eq,
                    99: self.hcf}
        self.memory = memory
        self.pointer = 0
        self.inputs = inputs
        self.outputs = outputs

    def step(self):
        try:
            while True:
                self.ops[self.memory[self.pointer] % 100]()
        except Yield:
            return True
        except Halt:
            return False

    def add(self):
        a, b = self.decode(2)
        self.memory[self.memory[self.pointer]] = a + b
        self.pointer += 1

    def mul(self):
        a, b = self.decode(2)
        self.memory[self.memory[self.pointer]] = a * b
        self.pointer += 1

    def input_(self):
        self.decode(0)
        self.memory[self.memory[self.pointer]] = self.inputs.pop(0)
        self.pointer += 1

    def output(self):
        a, = self.decode(1)
        self.outputs.append(a)
        raise Yield()

    def jnz(self):
        a, b = self.decode(2)
        if a != 0:
            self.pointer = b

    def jz(self):
        a, b = self.decode(2)
        if a == 0:
            self.pointer = b

    def lt(self):
        a, b = self.decode(2)
        self.memory[self.memory[self.pointer]] = 1 if a < b else 0
        self.pointer += 1

    def eq(self):
        a, b = self.decode(2)
        self.memory[self.memory[self.pointer]] = 1 if a == b else 0
        self.pointer += 1

    def hcf(self):
        raise Halt()

    def decode(self, count):
        mode = self.memory[self.pointer] // 100
        self.pointer += 1

        params = []
        for _ in range(count):
            value = self.memory[self.pointer]
            params.append(value if mode % 10 else self.memory[value])
            self.pointer += 1
            mode //= 10
        return params

def permutations(s):
    if s == []:
        yield []
    for i in range(len(s)):
        for p in permutations(s[:i] + s[i + 1:]):
            yield [s[i]] + p


## Part 1

In [20]:
def run(phases):
    queues = [[p] for p in phases] + [[]]
    amps = [VM(data.copy(), i, o)
            for i, o in zip(queues[:-1], queues[1:])]

    queues[0].append(0)
    for amp in amps:
        amp.step()
    return queues[-1].pop(0)

max(run(p) for p in permutations([4, 3, 2, 1, 0]))


87138

## Part 2

In [21]:
def run(phases):
    queues = [[p] for p in phases]
    amps = [VM(data.copy(), i, o)
            for i, o in zip(queues, queues[1:] + queues[:1])]

    queues[0].append(0)
    while all(a.step() for a in amps):
        pass
    return queues[0].pop(0)

max(run(p) for p in permutations([9, 8, 7, 6, 5]))


17279674